# Data Aggregation by Day

FiveThreeEight methodology:
- cap sample sizes at 5000
- if sample size isn't reported, use median sample size of polls from that polster (if no other info, use median sample size of all other polls
- sample size weighting - square root of poll's sample size / square root for median sample size for group

See `notes-p1.md` for details.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data-p1/polls_us_election_2016.csv")

df['startdate'] = pd.to_datetime(df['startdate'])
df['enddate'] = pd.to_datetime(df['enddate'])

df

,state,startdate,enddate,pollster,grade,samplesize,population,rawpoll_clinton,rawpoll_trump,rawpoll_johnson,rawpoll_mcmullin,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin
0,U.S.,2016-11-03,2016-11-06,ABC News/Washington Post,A+,2220.0,lv,47.00,43.00,4.00,NaN,45.20163,41.72430,4.626221,NaN
1,U.S.,2016-11-01,2016-11-07,Google Consumer Surveys,B,26574.0,lv,38.03,35.69,5.46,NaN,43.34557,41.21439,5.175792,NaN
2,U.S.,2016-11-02,2016-11-06,Ipsos,A-,2195.0,lv,42.00,39.00,6.00,NaN,42.02638,38.81620,6.844734,NaN
3,U.S.,2016-11-04,2016-11-07,YouGov,B,3677.0,lv,45.00,41.00,5.00,NaN,45.65676,40.92004,6.069454,NaN
4,U.S.,2016-11-03,2016-11-06,Gravis Marketing,B-,16639.0,rv,47.00,43.00,3.00,NaN,46.84089,42.33184,3.726098,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4203,Virginia,2016-09-16,2016-09-22,Ipsos,A-,452.0,lv,46.54,40.04,NaN,NaN,46.47852,40.48017,NaN,NaN
4204,Wisconsin,2016-08-04,2016-08-07,Marquette University,A,683.0,lv,47.00,34.00,9.00,NaN,48.74781,39.07778,4.705020,NaN
4205,Utah,2016-11-01,2016-11-07,Google Consumer Surveys,B,286.0,lv,21.33,35.05,9.99,NaN,26.65200,40.57738,9.705791,NaN
4206,Oregon,2016-10-21,2016-11-02,Ipsos,A-,446.0,lv,46.46,37.41,NaN,NaN,45.12949,37.10720,NaN,NaN


In [3]:
# df["grade"].value_counts()

In [4]:
grade_order = ["F","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"] # still included missing grades
df['grade'] = pd.Categorical(df['grade'], categories=grade_order[::-1], ordered=True)

In [5]:
df["population"].value_counts() # will probably just use likely voters, but keeping all for now in case

population
lv    3727
rv     418
v       42
a       21
Name: count, dtype: int64

In [6]:
df.isna().sum()

state                  0
startdate              0
enddate                0
pollster               0
grade                429
samplesize             1
population             0
rawpoll_clinton        0
rawpoll_trump          0
rawpoll_johnson     1409
rawpoll_mcmullin    4178
adjpoll_clinton        0
adjpoll_trump          0
adjpoll_johnson     1409
adjpoll_mcmullin    4178
dtype: int64

In [7]:
df = df.drop(columns=["rawpoll_johnson","rawpoll_mcmullin","adjpoll_johnson","adjpoll_mcmullin"]) # just looking at Clinton and Trump for now

In [8]:
pollster_medians = df.groupby('pollster')['samplesize'].median()
overall_median = float(df['samplesize'].median())

# imputation for sample size based on FiveThirtyEight methodology
def fill_na_with_median(row):

    if pd.isna(row['samplesize']):
        pollster_median = pollster_medians[row['pollster']]
        
        if pd.isna(pollster_median):
            return overall_median
        
        return pollster_median
    
    return row['samplesize']

df['samplesize'] = df.apply(fill_na_with_median, axis=1)

In [9]:
df['samplesizeadj'] = df['samplesize'].apply(lambda x: min(x, 5000)) # cap sample sizes at 5000

In [15]:
df.to_hdf('data-p1/polls-clean.h5', key='df', mode='w', format='t')
df.to_csv('data-p1/polls_clean.csv',index=False)

In [22]:
import datetime
df.iloc[0]['startdate'] - datetime.timedelta(days=1)

Timestamp('2016-11-02 00:00:00')

In [18]:
df.iloc[0]

state                                  U.S.
startdate               2016-11-03 00:00:00
enddate                 2016-11-06 00:00:00
pollster           ABC News/Washington Post
grade                                    A+
samplesize                           2220.0
population                               lv
rawpoll_clinton                        47.0
rawpoll_trump                          43.0
adjpoll_clinton                    45.20163
adjpoll_trump                       41.7243
samplesizeadj                        2220.0
Name: 0, dtype: object

In [14]:
def adjust_dates(group):
    group = group.sort_values(by='startdate').reset_index(drop=True)
    for i in range(len(group) - 1):
        if group.loc[i, 'enddate'] >= group.loc[i+1, 'startdate']:
            group.loc[i, 'enddate'] = group.loc[i+1, 'startdate'] - pd.Timedelta(days=1)
    return group

# Group by 'poll_name' and apply the function to adjust date ranges within each poll
df_adj = df.groupby(['pollster', 'state', 'population', 'grade'], group_keys=False,dropna=False).apply(adjust_dates)

C:\Users\mairi\AppData\Local\Temp\ipykernel_30772\3975376035.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_adj = df.groupby(['pollster', 'state', 'population', 'grade'], group_keys=False,dropna=False).apply(adjust_dates)
C:\Users\mairi\AppData\Local\Temp\ipykernel_30772\3975376035.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_adj = df.groupby(['pollster', 'state', 'population', 'grade'], group_keys=False,dropna=False).apply(adjust_dates)


In [17]:
df_adj.sort_values(by=['state','pollster','population','grade','startdate'])

,state,startdate,enddate,pollster,grade,samplesize,population,rawpoll_clinton,rawpoll_trump,adjpoll_clinton,adjpoll_trump,samplesizeadj
0,Alabama,2016-08-10,2016-08-16,Google Consumer Surveys,B,407.0,lv,19.55,54.53,26.33136,64.20842,407.0
1,Alabama,2016-08-17,2016-08-23,Google Consumer Surveys,B,414.0,lv,17.97,56.78,25.24864,65.91230,414.0
2,Alabama,2016-08-24,2016-08-30,Google Consumer Surveys,B,455.0,lv,17.11,56.17,24.57778,64.97036,455.0
3,Alabama,2016-08-31,2016-09-06,Google Consumer Surveys,B,358.0,lv,21.28,55.58,28.74781,64.18237,358.0
4,Alabama,2016-09-07,2016-09-13,Google Consumer Surveys,B,425.0,lv,19.02,56.78,26.40112,65.12639,425.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9,Wyoming,2016-10-30,2016-10-31,SurveyMonkey,C-,202.0,lv,22.00,58.00,20.66695,57.38503,202.0
10,Wyoming,2016-11-01,2016-11-07,SurveyMonkey,C-,213.0,lv,21.00,60.00,19.60153,59.33191,213.0
0,Wyoming,2016-08-09,2016-09-01,SurveyMonkey,C-,727.0,rv,21.00,57.00,23.26633,62.79952,727.0
0,Wyoming,2016-10-05,2016-10-11,University of Wyoming,NaN,722.0,lv,20.00,58.00,21.11059,60.54965,722.0


In [19]:
def split_to_individual_days(df):
    expanded_rows = []
    
    for idx, row in df.iterrows():
        day_range = pd.date_range(start=row['startdate'], end=row['enddate'])
        
        for day in day_range:
            expanded_row = row.copy()
            expanded_row['day'] = day
            expanded_rows.append(expanded_row)
    
    return pd.DataFrame(expanded_rows)

# # formula from FiveThirtyEight
# def weighted_average(group, value_column):
#     sqrt_sample_sizes = group['samplesizeadj'] ** 0.5 
#     median_sqrt_sample_size = group['samplesizeadj'].median() ** 0.5
#     weights = sqrt_sample_sizes / median_sqrt_sample_size
#     return (group[value_column] * weights).sum() / weights.sum()

# def aggregate_by_day(df):
#     agg_df = df.groupby(['pollster', 'state', 'population', 'day','grade'],dropna=False).apply(
#         lambda x: pd.Series({
#             'weighted_clinton': weighted_average(x, 'rawpoll_clinton'),
#             'weighted_trump': weighted_average(x, 'rawpoll_trump'),
#             'samplesize_total': x['samplesizeadj'].sum()
#         })
#     ).reset_index()
    
#     agg_df["c-minus-t"] = agg_df["weighted_clinton"] - agg_df["weighted_trump"]
#     return agg_df

# expanded_df = split_to_individual_days(df)

# df_agg = aggregate_by_day(expanded_df)

df_agg = split_to_individual_days(df_adj)

df_agg
 #20297

,state,startdate,enddate,pollster,grade,samplesize,population,rawpoll_clinton,rawpoll_trump,adjpoll_clinton,adjpoll_trump,samplesizeadj,day
0,Maryland,2016-09-27,2016-09-30,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-27
0,Maryland,2016-09-27,2016-09-30,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-28
0,Maryland,2016-09-27,2016-09-30,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-29
0,Maryland,2016-09-27,2016-09-30,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-30
0,Maryland,2016-03-30,2016-04-03,ABC News/Washington Post,A+,752.0,rv,63.0,28.0,60.84333,30.40937,752.0,2016-03-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,U.S.,2016-09-15,2016-09-19,icitizen,NaN,1000.0,rv,42.0,37.0,45.81087,42.71049,1000.0,2016-09-15
1,U.S.,2016-09-15,2016-09-19,icitizen,NaN,1000.0,rv,42.0,37.0,45.81087,42.71049,1000.0,2016-09-16
1,U.S.,2016-09-15,2016-09-19,icitizen,NaN,1000.0,rv,42.0,37.0,45.81087,42.71049,1000.0,2016-09-17
1,U.S.,2016-09-15,2016-09-19,icitizen,NaN,1000.0,rv,42.0,37.0,45.81087,42.71049,1000.0,2016-09-18


In [27]:
#df_agg.drop(columns=['startdate','enddate'],inplace=True)

df_agg['c_minus_t_raw'] = df_agg['rawpoll_clinton'] - df_agg['rawpoll_trump']
df_agg['c_minus_t_adj'] = df_agg['adjpoll_clinton'] - df_agg['adjpoll_trump']
df_agg

,state,pollster,grade,samplesize,population,rawpoll_clinton,rawpoll_trump,adjpoll_clinton,adjpoll_trump,samplesizeadj,day,c_minus_t_raw,c_minus_t_adj
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-27,36.0,33.96959
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-28,36.0,33.96959
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-29,36.0,33.96959
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-30,36.0,33.96959
0,Maryland,ABC News/Washington Post,A+,752.0,rv,63.0,28.0,60.84333,30.40937,752.0,2016-03-30,35.0,30.43396
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,U.S.,icitizen,NaN,1000.0,rv,42.0,37.0,45.81087,42.71049,1000.0,2016-09-15,5.0,3.10038
1,U.S.,icitizen,NaN,1000.0,rv,42.0,37.0,45.81087,42.71049,1000.0,2016-09-16,5.0,3.10038
1,U.S.,icitizen,NaN,1000.0,rv,42.0,37.0,45.81087,42.71049,1000.0,2016-09-17,5.0,3.10038
1,U.S.,icitizen,NaN,1000.0,rv,42.0,37.0,45.81087,42.71049,1000.0,2016-09-18,5.0,3.10038


In [28]:
df_agg.to_hdf('data-p1/agg_polls_by_day.h5', key='df', mode='w', format='t') # h5 format to preserve data types (categorical, datetime, etc)
df_agg.to_csv('data-p1/agg_polls_by_day.csv', index=False) # csv format in case anyone wants to use

### Electoral College

In [29]:
df_mn = df_adj[(df_adj['state'] == 'Maine') | (df_adj['state'] == 'Nebraska')]
df_mn['state'] = [i.split(' ')[0] for i in df_mn['state']]
df_mn['state'] = [f'{i} State' for i in df_mn['state']]
df_mn

C:\Users\mairi\AppData\Local\Temp\ipykernel_30772\853714716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mn['state'] = [i.split(' ')[0] for i in df_mn['state']]
C:\Users\mairi\AppData\Local\Temp\ipykernel_30772\853714716.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mn['state'] = [f'{i} State' for i in df_mn['state']]


,state,startdate,enddate,pollster,grade,samplesize,population,rawpoll_clinton,rawpoll_trump,adjpoll_clinton,adjpoll_trump,samplesizeadj
0,Maine State,2016-03-04,2016-03-10,Critical Insights,C,610.0,lv,43.00,34.00,41.56599,36.13324,610.0
0,Maine State,2016-09-02,2016-09-05,Emerson College,B,800.0,lv,44.00,35.00,46.92659,36.99561,800.0
1,Maine State,2016-10-28,2016-10-30,Emerson College,B,750.0,lv,46.00,42.00,46.20038,40.92931,750.0
0,Nebraska State,2016-09-25,2016-09-26,Emerson College,B,700.0,lv,29.30,55.70,31.15957,56.71930,700.0
0,Maine State,2016-08-10,2016-08-16,Google Consumer Surveys,B,121.0,lv,45.25,17.56,52.73279,28.41731,121.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,Maine State,2016-06-15,2016-06-21,University of New Hampshire,B+,475.0,lv,42.00,35.00,41.76123,38.11454,475.0
1,Maine State,2016-09-15,2016-09-20,University of New Hampshire,B+,513.0,lv,40.00,36.00,42.36707,40.67970,513.0
2,Maine State,2016-10-20,2016-10-25,University of New Hampshire,B+,670.0,lv,48.00,37.00,47.81695,40.65277,670.0
0,Maine State,2016-10-04,2016-11-06,YouGov,B,437.0,lv,38.70,34.00,39.56763,36.74421,437.0


In [30]:
# expanded_df_mn = split_to_individual_days(df_mn)
# df_agg_mn = aggregate_by_day(expanded_df_mn)
df_agg_mn = split_to_individual_days(df_mn)
df_agg_mn

,state,startdate,enddate,pollster,grade,samplesize,population,rawpoll_clinton,rawpoll_trump,adjpoll_clinton,adjpoll_trump,samplesizeadj,day
0,Maine State,2016-03-04,2016-03-10,Critical Insights,C,610.0,lv,43.0,34.0,41.56599,36.13324,610.0,2016-03-04
0,Maine State,2016-03-04,2016-03-10,Critical Insights,C,610.0,lv,43.0,34.0,41.56599,36.13324,610.0,2016-03-05
0,Maine State,2016-03-04,2016-03-10,Critical Insights,C,610.0,lv,43.0,34.0,41.56599,36.13324,610.0,2016-03-06
0,Maine State,2016-03-04,2016-03-10,Critical Insights,C,610.0,lv,43.0,34.0,41.56599,36.13324,610.0,2016-03-07
0,Maine State,2016-03-04,2016-03-10,Critical Insights,C,610.0,lv,43.0,34.0,41.56599,36.13324,610.0,2016-03-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Nebraska State,2016-10-04,2016-11-06,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-02
0,Nebraska State,2016-10-04,2016-11-06,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-03
0,Nebraska State,2016-10-04,2016-11-06,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-04
0,Nebraska State,2016-10-04,2016-11-06,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-05


In [31]:
df_agg_all = pd.concat([df_agg, df_agg_mn])
df_agg_all

,state,pollster,grade,samplesize,population,rawpoll_clinton,rawpoll_trump,adjpoll_clinton,adjpoll_trump,samplesizeadj,day,c_minus_t_raw,c_minus_t_adj,startdate,enddate
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-27,36.0,33.96959,NaT,NaT
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-28,36.0,33.96959,NaT,NaT
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-29,36.0,33.96959,NaT,NaT
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-30,36.0,33.96959,NaT,NaT
0,Maryland,ABC News/Washington Post,A+,752.0,rv,63.0,28.0,60.84333,30.40937,752.0,2016-03-30,35.0,30.43396,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Nebraska State,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-02,NaN,NaN,2016-10-04,2016-11-06
0,Nebraska State,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-03,NaN,NaN,2016-10-04,2016-11-06
0,Nebraska State,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-04,NaN,NaN,2016-10-04,2016-11-06
0,Nebraska State,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-05,NaN,NaN,2016-10-04,2016-11-06


In [32]:
electoral_college = {
    "Alabama": 9,
    "Alaska": 3,
    "Arizona": 11,
    "Arkansas": 6,
    "California": 55,
    "Colorado": 9,
    "Connecticut": 7,
    "Delaware": 3,
    "District of Columbia": 3,
    "Florida": 29,
    "Georgia": 16,
    "Hawaii": 4,
    "Idaho": 4,
    "Illinois": 20,
    "Indiana": 11,
    "Iowa": 6,
    "Kansas": 6,
    "Kentucky": 8,
    "Louisiana": 8,
    "Maine State": 2,
    "Maine CD-1": 1,
    "Maine CD-2": 1,
    "Maryland": 10,
    "Massachusetts": 11,
    "Michigan": 16,
    "Minnesota": 10,
    "Mississippi": 6,
    "Missouri": 10,
    "Montana": 3,
    "Nebraska State": 2,
    "Nebraska CD-1": 1,
    "Nebraska CD-2": 1,
    "Nebraska CD-3": 1,
    "Nevada": 6,
    "New Hampshire": 4,
    "New Jersey": 14,
    "New Mexico": 5,
    "New York": 29,
    "North Carolina": 15,
    "North Dakota": 3,
    "Ohio": 18,
    "Oklahoma": 7,
    "Oregon": 7,
    "Pennsylvania": 20,
    "Rhode Island": 4,
    "South Carolina": 9,
    "South Dakota": 3,
    "Tennessee": 11,
    "Texas": 38,
    "Utah": 6,
    "Vermont": 3,
    "Virginia": 13,
    "Washington": 12,
    "West Virginia": 5,
    "Wisconsin": 10,
    "Wyoming": 3
}


In [33]:
df_agg_all['electoral_college'] = df_agg_all['state'].map(electoral_college)
df_agg_all

,state,pollster,grade,samplesize,population,rawpoll_clinton,rawpoll_trump,adjpoll_clinton,adjpoll_trump,samplesizeadj,day,c_minus_t_raw,c_minus_t_adj,startdate,enddate,electoral_college
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-27,36.0,33.96959,NaT,NaT,10.0
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-28,36.0,33.96959,NaT,NaT,10.0
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-29,36.0,33.96959,NaT,NaT,10.0
0,Maryland,ABC News/Washington Post,A+,706.0,lv,63.0,27.0,62.70862,28.73903,706.0,2016-09-30,36.0,33.96959,NaT,NaT,10.0
0,Maryland,ABC News/Washington Post,A+,752.0,rv,63.0,28.0,60.84333,30.40937,752.0,2016-03-30,35.0,30.43396,NaT,NaT,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Nebraska State,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-02,NaN,NaN,2016-10-04,2016-11-06,2.0
0,Nebraska State,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-03,NaN,NaN,2016-10-04,2016-11-06,2.0
0,Nebraska State,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-04,NaN,NaN,2016-10-04,2016-11-06,2.0
0,Nebraska State,YouGov,B,506.0,lv,32.3,44.9,33.14527,47.34476,506.0,2016-11-05,NaN,NaN,2016-10-04,2016-11-06,2.0


In [34]:
df_agg_all.to_hdf('data-p1/electoral_college_agg_polls_by_day.h5', key='df', mode='w', format='t') # h5 format to preserve data types (categorical, datetime, etc)
df_agg_all.to_csv('data-p1/electoral_college_agg_polls_by_day.csv', index=False) # csv format in case anyone wants to use